In [51]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.nn.functional as F

from torch.autograd import Variable as V
from torch import Tensor as T
from IPython.display import display

In [90]:
class MyRNN(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim = 64):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_in = nn.Linear(in_dim + hidden_dim, hidden_dim)
        self.layer_hidden = nn.Linear(hidden_dim, hidden_dim)
        self.layer_out = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, *X):
        n = X[0].size(0)
        h = V(torch.zeros(n, self.hidden_dim))
        
        for item in X:
            new_input = F.relu(self.layer_in(torch.cat([h, item], 1)))
            h = F.tanh(self.layer_hidden(new_input))
        
        return F.log_softmax(self.layer_out(h), -1)

In [91]:
rnn = MyRNN(5, 2, 16)

In [92]:
rnn(V(T([[1,0,1,1,0]])))

Variable containing:
-0.8292 -0.5734
[torch.FloatTensor of size 1x2]

In [93]:
X = np.random.binomial(1, .5, (10000, 5))
y = X.sum(axis = 1) % 2
print(X[0], y[0])

[0 0 0 0 1] 1


In [94]:
Xtrain, Xtest, ytrain, ytest = X[:8000], X[8000:], y[:8000], y[8000:]

In [95]:
loss_func = F.nll_loss
optim = torch.optim.Adam(rnn.parameters())

In [97]:
XtrainV, XtestV, ytrainV, ytestV = [V(torch.from_numpy(item.astype(np.float32))) for item in [Xtrain, Xtest, ytrain, ytest]]

In [134]:
for i in range(10):
    y_pred = rnn(XtrainV)
    loss = loss_func(y_pred, V(torch.LongTensor(ytrain.astype(np.int64))))
    optim.zero_grad()
    loss.backward()
    optim.step()
    test_preds = rnn(XtestV).data.numpy()
    display(pd.crosstab(ytest, test_preds.argmax(axis = 1)))

col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


col_0,0,1
row_0,,
0,1017,0
1,0,983


In [136]:
rnn(V(T([[1,1,1,1,1]])))

Variable containing:
-1.5793 -0.2308
[torch.FloatTensor of size 1x2]